In [11]:
import pandas as pd
import seaborn as sns
import numpy as np
import regex as re
import matplotlib.pyplot as plt
pd.options.display.max_columns = None
df = df = pd.read_csv("ataques_enproceso.csv")
#del df["unnamed:_0"]
df.head()

,Unnamed: 0,case_number,year,type,country,area,location,activity,name,sex_,age,time,species_,fatal_(y/n),injury,date
0,0,2018.06.25,2018.0,Boating,usa,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,18h00,White shark,N,"No injury to occupant, outrigger canoe and pad...",25-Jun-2018
1,1,2018.06.18,2018.0,Unprovoked,usa,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,14h00 -15h00,NaN,N,Minor injury to left thigh,18-Jun-2018
2,2,2018.06.09,2018.0,Invalid,usa,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,07h45,NaN,N,Injury to left lower leg from surfboard skeg,09-Jun-2018
3,3,2018.06.08,2018.0,Unprovoked,australia,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,NaN,2 m shark,N,Minor injury to lower leg,08-Jun-2018
4,4,2018.06.04,2018.0,Provoked,mexico,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,NaN,"Tiger shark, 3m",N,Lacerations to leg & hand shark PROVOKED INCIDENT,04-Jun-2018


In [3]:
df.dtypes

Unnamed: 0       int64
case_number     object
year           float64
type            object
country         object
area            object
location        object
activity        object
name            object
sex_            object
age             object
time            object
species_        object
fatal_(y/n)     object
injury          object
date            object
dtype: object

Antes de seguir, recordamos las preguntas que nos planteamos al principio del _pair programming_** de EDA para dirigir nuestro análisis.**

-¿Es Australia es el sitio más peligroso y letal para estar relajada en la playa?

-¿Cuál es el rango de edad que sufre la mayoría de los ataques?

-Independientemente de la edad, sufren los hombres más ataques que las mujeres?

-¿En qué mes ocurren más ataques?

-¿Cuál es la relación entre la especie y el tipo de ataque (si es fatal o no)?

-¿Cómo han evolucionado los ataques a lo largo del tiempo?

De todo esto, nos damos cuenta que solo tenemos que limpiar algunas de las columnas, en concreto age, species, country, fatal, year, sex. Si reducimos esto a una tabla para saber que tenemos ya limpito y que no. **Actualizamos esta tabla ya que en el ejercicio de pair de Limpieza I ya dejamos algunas columnas limpitas:

Por lo tanto, en este ejercicio nos queda por limpiar las columnas de age y species, así que manos a la obra. Vamos a ver que podemos hacer para cada una de las columnas y os dejaremos algunas pistas 😉

1 -**Columna de species:**

 si exploramos esta columna en detalle nos podemos dar cuenta que tenemos muchos valores únicos y esto hace que sea muy difícil trabajar con esta columna. Lo que vamos a hacer es clasificar los tiburones en 5 especies diferentes, las más comunes, que incluyen el tiburón blanco (White), el tiburón tigre (Tiger ), el tiburón gris (Grey), el tiburón limón (Lemon) y el tiburón toro (Bull). El resto de las especies las incluiremos en un único grupo que podremos llamar "Unspecified". Ahora nos podemos sentir un poco abrumadas y no saber como enfrentarnos a este reto, pero don't worry, os dejamos por aquí unas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo.
Los valores de las columnas son strings por lo que podremos usar regex para buscar palabras clave en cada celda y asignarlo a una de las categorías que hemos definido previamente.

 *imaginemos que el valor de una celda es el siguiente*

*'White shark, 3.5 m'*

* tendremos que buscar el patrón de regex que nos permita extraer White shark de ese string y que nos devuelva solo White shark. *
* Un patrón que podriamos usar es: *

*patron_blanco = r".*[Ww](hite|HITE).*" # esto podría ser así porque puede estar en mayúsculas o en minúsculas. *

* de la misma forma que hemos sacado el patron para el tiburón blanco, tendremos que sacar los patrones para las otras 4 especies que queremos "encontrar". *

Tendremos que crearnos una función que aplicaremos sobre nuestra columna species_ para que nos devuelva una nueva columna con los valores clasificados en función de los patrones de regex que hayamos definido.

In [4]:
patron_blanco =r".*[Ww](hite|HITE).*"
patron_tigre =r".*[Tt](iger|IGER).*"
patron_gris = r".*[Gg](rey|REY).*"
patron_limon =r".*[Ll](emon|EMON).*"
patron_torito = r".*[Bb](ull|ULL).*"


In [5]:
def funcion (columna):
    
    try:
        if re.findall(patron_blanco,columna) :
            return "white_shark"
        elif re.findall(patron_tigre,columna) :
            return "tiger_shark"
        elif re.findall(patron_gris,columna) :
            return "grey_shark"
        elif re.findall(patron_limon,columna) :
            return "lemon_shark"
        elif re.findall(patron_torito,columna) :
            return "bull_shark"
        else:
            return "unspecified"
    except:
        return "unspecified"

In [6]:
df["species_"] = df["species_"].apply(funcion)
df.head(5)

,Unnamed: 0,case_number,year,type,country,area,location,activity,name,sex_,age,time,species_,fatal_(y/n),injury,date
0,0,2018.06.25,2018.0,Boating,usa,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,18h00,white_shark,N,"No injury to occupant, outrigger canoe and pad...",25-Jun-2018
1,1,2018.06.18,2018.0,Unprovoked,usa,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,14h00 -15h00,unspecified,N,Minor injury to left thigh,18-Jun-2018
2,2,2018.06.09,2018.0,Invalid,usa,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,07h45,unspecified,N,Injury to left lower leg from surfboard skeg,09-Jun-2018
3,3,2018.06.08,2018.0,Unprovoked,australia,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,NaN,unspecified,N,Minor injury to lower leg,08-Jun-2018
4,4,2018.06.04,2018.0,Provoked,mexico,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,NaN,tiger_shark,N,Lacerations to leg & hand shark PROVOKED INCIDENT,04-Jun-2018


**Columna de age: es una columna de tipo string pero debería ser de tipo integer. Además, en esta columna nos vamos a encontrar con algunos errores tipográficos, estos incluyen:**

-Edad en formato string

-Edades separadas por &, or, to, >

-Edades con ?

In [7]:
#Tenemos que evitar los Nans porque sino no nos deja convertirlos en int.

In [33]:
df_2 = pd.DataFrame(df) 
del df_2["age_limpio"]
df_2.head(2)

,Unnamed: 0,case_number,year,type,country,area,location,activity,name,sex_,age,time,species_,fatal_(y/n),injury,date
0,0,2018.06.25,2018.0,Boating,usa,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,18h00,White shark,N,"No injury to occupant, outrigger canoe and pad...",25-Jun-2018
1,1,2018.06.18,2018.0,Unprovoked,usa,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,14h00 -15h00,NaN,N,Minor injury to left thigh,18-Jun-2018


In [32]:
patron_age = "(0-9)"#necesito un patron regex para sacar numeros como si fuesen strings, ya que no son int.

def limpio (col):
    try:
        if re.findall(patron_age, col):
            pass
        else:
            return "0"
    except:
        return "0"


df_2.insert(11,"age_limpio",limpio("age"),True)
df_2


,Unnamed: 0,case_number,year,type,country,area,location,activity,name,sex_,age,age_limpio,time,species_,fatal_(y/n),injury,date
0,0,2018.06.25,2018.0,Boating,usa,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,0,18h00,White shark,N,"No injury to occupant, outrigger canoe and pad...",25-Jun-2018
1,1,2018.06.18,2018.0,Unprovoked,usa,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,0,14h00 -15h00,NaN,N,Minor injury to left thigh,18-Jun-2018
2,2,2018.06.09,2018.0,Invalid,usa,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,0,07h45,NaN,N,Injury to left lower leg from surfboard skeg,09-Jun-2018
3,3,2018.06.08,2018.0,Unprovoked,australia,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,0,NaN,2 m shark,N,Minor injury to lower leg,08-Jun-2018
4,4,2018.06.04,2018.0,Provoked,mexico,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,0,NaN,"Tiger shark, 3m",N,Lacerations to leg & hand shark PROVOKED INCIDENT,04-Jun-2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6283,6283,ND.0001,0.0,Unprovoked,ceylon (sri lanka),Eastern Province,"Below the English fort, Trincomalee",Swimming,male,M,15,0,NaN,NaN,Y,"FATAL. ""Shark bit him in half, carrying away t...",1845-1853
6284,6284,2017.01.08.R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,"No attack, shark made a threat display",Reported 08-Jan-2017
6285,6285,1836.08.19.R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,Y,FATAL,Reported 19-Aug-1836
6286,6286,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


In [8]:
df["age"] = df["age"].astype("Int64", errors = "ignore")
df.dtypes

Unnamed: 0       int64
case_number     object
year           float64
type            object
country         object
area            object
location        object
activity        object
name            object
sex_            object
age             object
time            object
species_        object
fatal_(y/n)     object
injury          object
date            object
dtype: object

Vamos con algunas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo:

-Primero tendremos que eliminar todos esos símbolos especiales que nos aparecen. De nuevo, podremos usar regex para extraer unicamente los números que es lo que nos interesa. Usar este regex en una función para sacar solo los números.

-Puede que os salga un error similar a este:

TypeError: expected string or bytes-like object

Para solucionar este problema, antes de nada tendréis que ejecutar este código para que no os de error:

In [ ]:
df['nombre_columna'] = df['nombre_columna'].astype(str)

-Una vez que hayáis extraido los números, os daréis cuenta que hay celdas que tienen más de una edad. Tendréis que decidir que hacer en esos casos. Os dejamos por aquí una posible opción usando un método de Pandas que os puede resultar super útil. El método explode, aqui.

-Por último cambiad el tipo de la columna de string a integer.

3- Guarda el csv con las columnas limpias para seguir trabajando con este dataframe limpio.